In [ ]:
from player.parse import *
from player.state import *
from player.model import *
from player.ndlstm import separable_lstm
from player.constants import MAX_BOARD_SIZE, FEATURE_SIZE, OUTPUT_SIZE


import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.contrib.rnn import GRUCell

In [2]:
m = HaliteModel()

In [3]:
train, test = train_test_split('../training_data/', 200, 0.2)
dg = data_gen2(train[0], 3)

In [4]:

map_size

56

In [5]:
states = parse_winner('../training_data/ts2018-halite-3-gold-replays_replay-20181021-025544+0000-1540090539-56-56-772361')

In [11]:
import numpy as np
i = 3
print(states[i].get_ship_moves())
print(states[i].ships)
print(states[i].frame[14][29])
features = states[i].get_feature_map()
print(features[14][29])

[(4, (14, 29))]
{1: Ship(id=1, Position(14, 29), cargo=0 halite)}
110
[0.11  0.    0.    0.    0.    0.006]


In [15]:
map_size, features, mask_slice, moves = next(dg)
print(mask_slice)
print(features[0,43,27,:])
print(features[1,42,28,:])
print(features[2,46,30,:])


[[ 0 43 28]
 [ 1 45 33]
 [ 2 46 32]]
[0.081 0.029 0.    0.    0.    0.086]
[0.015 0.    0.    0.    0.    0.086]
[0.167 0.859 0.    0.    0.    0.086]


In [37]:
print(mask_slice)
print(features[0,43,28,:])
print(features[1,45,33,:])
print(features[2,46,32,:])

sess = tf.Session()


x = tf.placeholder(tf.float32, [5])
s = tf.placeholder(tf.float32, [None, 5])
y = tf.multiply(s, tf.broadcast_to(x, tf.shape(s)))
tf.global_variables_initializer()

sess.run([y], feed_dict = {x:[1,1,1,1,0.1], s:np.ones([12, 5])})

[[ 0 43 28]
 [ 1 45 33]
 [ 2 46 32]]
[0.054 0.005 0.    0.    0.    0.086]
[0.058 0.873 0.    0.    0.    0.086]
[0.182 0.964 0.    0.    0.    0.086]


[array([[1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1],
        [1. , 1. , 1. , 1. , 0.1]], dtype=float32)]

In [38]:
def data_gen3(filename, batch_size):
    feature_list = []
    idx_list = []
    move_list = []
    included_states = []
    states = parse_winner(filename)
    first_state = states[0]
    map_size = first_state.map_size
    for i in np.random.permutation(len(states))[:20]:
        s = states[i]
        included_states.append(s)
        feature_map = s.get_feature_map()
        moves = s.get_ship_moves()
        for j in range(len(moves)):
            move_tuple = moves[j]
            new_idx = [len(feature_list), move_tuple[1][0], move_tuple[1][1]]
            feature_list.append(feature_map)
            idx_list.append(new_idx)
            move_list.append(move_tuple[0])
            if len(feature_list) >= batch_size:
                f = np.stack(feature_list, axis=0)
                mi = np.stack(idx_list, axis=0)
                mo = np.stack(move_list, axis=0)
                yield map_size, f, mi, mo, included_states
                feature_list = []
                idx_list = []
                move_list = []
                included_states = []
    if len(feature_list):
        f = np.stack(feature_list, axis=0)
        mi = np.stack(idx_list, axis=0)
        mo = np.stack(move_list, axis=0)
        yield map_size, f, mi, mo, included_states

In [39]:
si, f, mi, mo, sts = next(data_gen3('../training_data/ts2018-halite-3-gold-replays_replay-20181021-025544+0000-1540090539-56-56-772361', 3))

In [46]:
print(mi)
print(mo)
print(sts[0].ships)

[[ 0  7 40]
 [ 1 11 20]
 [ 2 18 32]]
[3 4 0]
{12: Ship(id=12, Position(7, 40), cargo=983 halite), 14: Ship(id=14, Position(11, 20), cargo=42 halite), 15: Ship(id=15, Position(18, 32), cargo=905 halite), 17: Ship(id=17, Position(4, 40), cargo=723 halite), 21: Ship(id=21, Position(55, 20), cargo=514 halite), 22: Ship(id=22, Position(6, 37), cargo=107 halite), 24: Ship(id=24, Position(11, 32), cargo=1 halite), 27: Ship(id=27, Position(16, 41), cargo=224 halite), 28: Ship(id=28, Position(10, 19), cargo=104 halite), 3: Ship(id=3, Position(16, 40), cargo=197 halite), 31: Ship(id=31, Position(8, 17), cargo=220 halite), 33: Ship(id=33, Position(6, 40), cargo=939 halite), 35: Ship(id=35, Position(1, 36), cargo=857 halite), 39: Ship(id=39, Position(6, 21), cargo=875 halite), 42: Ship(id=42, Position(0, 19), cargo=969 halite), 43: Ship(id=43, Position(2, 27), cargo=34 halite), 44: Ship(id=44, Position(52, 23), cargo=825 halite), 46: Ship(id=46, Position(2, 39), cargo=898 halite), 48: Ship(id=48, 

In [48]:
print(sts[0].frame[7][40])
print(sts[0].frame[11][20])
print(sts[0].frame[18][32])

6
9
4


In [49]:
print(f[0,7,40,:])
print(f[1,11,20,:])
print(f[2,18,32,:])

[0.006 0.983 0.    0.    0.    0.894]
[0.009 0.042 0.    0.    0.    0.894]
[0.004 0.905 0.    0.    0.    0.894]
